In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import time
from PIL import Image
import torch
import torchmetrics
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from pprint import pformat
from collections import defaultdict
from torchvision import datasets, transforms
from torch.utils.data.dataset import Dataset
from torchvision.datasets.utils import download_url, extract_archive

In [2]:
root = './'
filename = 'data.zip'
archive = os.path.join(root, filename)
data_folder = os.path.join(root, 'data/images')
if (not os.path.exists(data_folder)):
    extract_archive(archive, './data', False)
